<a href="https://colab.research.google.com/github/tryfasting/deeplearning-pytorch-textbook/blob/main/11-03%20DTM%EA%B3%BC%20TF-IDF%20%ED%96%89%EB%A0%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 서로 다른 문서들의 BoW들을 결합한 DTM(Document-Term Matrix, 문서 단어 행렬) 표현 방법을 배워보자.

### 1. DTM(Document-Term Matrix, 문서 단어 행렬)의 표기법

DTM이란 다수의 문서에서 등장하는 각 단어들의 빈도를 행렬로 표현한 것이다.  
즉, 각 문서에 대한 BoW를 하나의 행렬로 만든 것이다.

### 2. DTM의 한계

#### 1) 희소 표현
원-핫 벡터와 마찬가지로, 공간적 낭비와 계산 리소스를 증가시킬 수 있다. 전체 단어 집합만큼의 크기를 갖기 때문에, 코퍼스가 방대한 데이터라면 문서 벡터의 차원은 수만 이상이 될 것이다.  

원-핫 벡터나 DTM과 같은 대부분의 값이 0인 표현을 희소 벡터(sparse vector) 또는 희소 행렬(sparse matrix)라고 부르는데, 희소 벡터는 많은 양의 저장 공간과 높은 계산 복잡도를 요구한다. 이러한 이유로 전처리를 통해 단어 집합의 크기를 줄이는 일이 BoW 표현을 활용하는 모델에서 중요해진다.  
전처리는 구두점, 빈도수가 낮은 단어, 불용어를 제거하고,  
어간이나 표제어 추출로 단어를 정규화하는  
방법이 있겠다. 이 방법으로 단어 집합 크기를 줄일 수 있다.

#### 2) 단순 빈도 수 기반 접근
예컨대 단순 빈도 수로 접근하면, 'the'가 동일하게 빈도 수가 높다고 해서 문서1, 문서2, 문서3 등등이 유사한 문서라고 판단해서는 안된다.  

DTM에 불용어와 중요한 단어에 가중치를 주는 방법은 없을까?  
그것이 TF-IDF이다. 많은 경우에서 DTM보다 더 좋은 성능을 얻을 수 있다.

### 3. TF-IDF(단어 빈도-역 문서 빈도, Term Frequency-Inverse Document Frequency)

TF-IDF는 단어의 빈도와 역문서 빈도를 사용하여 DTM 내의 각 단어들마다 중요한 정도를 가중치로 주는 방법이다.

#### 1) tf(d,t) : 특정 문서 d에서의 특정 단어 t의 등장 횟수

#### 2) df(t) : 특정 단어 t가 등장한 문서의 수.

#### 3) idf(t) : df(t)에 반비례하는 수.
여러 문서에서 등장한 단어의 가중치를 낮추는 역할을 한다.

IDF는 DF에 log와 분모에 1을 더한 후 역수를 취한다.  
총 문서 수 n이 커질수록, IDF 값은 기하급수적으로 커지기 때문이다.   
또 다른 직관적인 설명은 다음과 같다.  
불용어와 비교적 자주 쓰이지 않는 단어의 빈도 수 차이는 수십 배가 난다.  
그런데 비교적 자주 쓰이지 않은 단어 조차 희귀 단어와 비교하면 최소 수백 배 차이가 난다.  
따라서 log를 씌워주지 않으면, 희귀 단어에 엄청난 가중치가 부여될 수 있다.  
log를 씌워주면 이러한 격차를 줄이는 효과가 있다.

TF-IDF는  
**모든 문서에서 자주 등장하는 단어**는 **중요도가 낮다**고 판단하며,  
**특정 문서에서만 자주 등장하는 단어**는 **중요도가 높다**고 판단한다.

TF-IDF값이 낮으면 중요도가 낮은 것이며,    
TF-IDF값이 높으면 중요도가 높은 것이다.

### 4. 파이썬으로 TF-IDF 직접 구현하기

In [1]:
import pandas as pd # 데이터프레임 사용을 위해서
from math import log # IDF 계산을 위해

docs = [
  '먹고 싶은 사과',
  '먹고 싶은 바나나',
  '길고 노란 바나나 바나나',
  '저는 과일이 좋아요'
]

vocab = list(set(w for doc in docs for w in doc.split()))
vocab.sort()

In [2]:
vocab

['과일이', '길고', '노란', '먹고', '바나나', '사과', '싶은', '저는', '좋아요']

TF, IDF, TF-IDF 값을 구하는 함수를 구현한다.

In [3]:
# 총 문서의 수
N = len(docs)

def tf(t,d):
    return d.count(t)

def idf(t):
    df = 0
    for doc in docs:
        df += t in doc
    return log(N/(df+1))

def tfidf(t,d):
    return tf(t,d) * idf(t)


TF를 구해보자.  
즉, DTM을 데이터프레임에 저장하여 출력하자.  

In [4]:
result = []

# 각 문서에 대해서 아래 연산을 반복
for i in range(N):
    result.append([])
    d = docs[i]
    for j in range(len(vocab)):
        t = vocab[j]
        result[-1].append(tf(t,d))

tf_ = pd.DataFrame(result, columns = vocab)

In [5]:
result

[[0, 0, 0, 1, 0, 1, 1, 0, 0],
 [0, 0, 0, 1, 1, 0, 1, 0, 0],
 [0, 1, 1, 0, 2, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 1, 1]]

In [6]:
tf_

,과일이,길고,노란,먹고,바나나,사과,싶은,저는,좋아요
0,0,0,0,1,0,1,1,0,0
1,0,0,0,1,1,0,1,0,0
2,0,1,1,0,2,0,0,0,0
3,1,0,0,0,0,0,0,1,1


각 단어에 대한 IDF 값을 구해보자.

In [7]:
result = []

for j in range(len(vocab)):
    t = vocab[j]
    result.append(idf(t))

idf_ = pd.DataFrame(result, index=vocab, columns=['IDF'])
idf_

,IDF
과일이,0.693147
길고,0.693147
노란,0.693147
먹고,0.287682
바나나,0.287682
사과,0.693147
싶은,0.287682
저는,0.693147
좋아요,0.693147


TF-IDF 행렬로 출력해보자.

In [8]:
result = []

for i in range(N):
    result.append([])
    d = docs[i]

    for j in range(len(vocab)):
        t = vocab[j]
        result[-1].append(tfidf(t,d))

tfidf_ = pd.DataFrame(result, columns = vocab)
tfidf_

,과일이,길고,노란,먹고,바나나,사과,싶은,저는,좋아요
0,0.000000,0.000000,0.000000,0.287682,0.000000,0.693147,0.287682,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.287682,0.287682,0.000000,0.287682,0.000000,0.000000
2,0.000000,0.693147,0.693147,0.000000,0.575364,0.000000,0.000000,0.000000,0.000000
3,0.693147,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.693147,0.693147


### 5. 사이킷런을 이용한 DTM과 TF-IDF 실습

BoW를 설명하며 배운 CountVectorizer를 사용하면 DTM을 만들 수 있다.

In [9]:
from sklearn.feature_extraction.text import CountVectorizer

corpus = [
    'you know I want your love',
    'I like you',
    'what should I do',
]

vector = CountVectorizer()

# 코퍼스로부터 각 단어의 빈도수를 기록
print(vector.fit_transform(corpus).toarray())

# 각 단어와 맵핑된 인덱스 출력
print(vector.vocabulary_)


[[0 1 0 1 0 1 0 1 1]
 [0 0 1 0 0 0 0 1 0]
 [1 0 0 0 1 0 1 0 0]]
{'you': 7, 'know': 1, 'want': 5, 'your': 8, 'love': 3, 'like': 2, 'what': 6, 'should': 4, 'do': 0}


사이킷런은 TF-IDF를 자동 계산해주는 TfidVectorizer를 제공한다.  
사이킷런은 위에서 배웠던 식에서 조정된 식을 사용한다.

In [ ]:
!pip install

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer

corpus = [
    'you know I want your love',
    'I like you',
    'what should I do',
]

tfidfv = TfidfVectorizer().fit(corpus)
print(tfidfv.transform(corpus).toarray())
print(tfidfv.vocabulary_)

[[0.         0.46735098 0.         0.46735098 0.         0.46735098
  0.         0.35543247 0.46735098]
 [0.         0.         0.79596054 0.         0.         0.
  0.         0.60534851 0.        ]
 [0.57735027 0.         0.         0.         0.57735027 0.
  0.57735027 0.         0.        ]]
{'you': 7, 'know': 1, 'want': 5, 'your': 8, 'love': 3, 'like': 2, 'what': 6, 'should': 4, 'do': 0}
